In [13]:
import os
import re
import time
import requests
from moviepy.editor import *
from gtts import gTTS

# Set your Pexels API key
PEXELS_API_KEY = "32TwCMSUffppd6DdqjWcYSjGXY8Y2E4mTle88MAe5UEcrHxl4eASZxaO"

# Ensure output directories exist
os.makedirs("audio", exist_ok=True)
os.makedirs("videos", exist_ok=True)

# Storyboard input directly in the code
storyboard_text = """

"""

# Function to extract scenes from structured text
def parse_storyboard(text):
    scenes = []
    matches = re.findall(r"\*\*Scene \d+ \(\d+-\d+ seconds\)\*\*\s+- \*\*Visual:\*\* (.+?)\s+- \*\*Audio:\*\* \"(.+?)\"", text, re.DOTALL)

    for i, (visual, audio) in enumerate(matches):
        if visual.strip() and audio.strip():
            scenes.append({"scene": f"Scene {i+1}", "visual": visual.strip(), "audio": audio.strip()})

    return scenes

# Function to fetch video from Pexels API
def fetch_pexels_video(query):
    url = "https://api.pexels.com/videos/search"
    headers = {"Authorization": PEXELS_API_KEY}
    params = {"query": query, "per_page": 1}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        if data.get("videos"):
            return data["videos"][0]["video_files"][0]["link"]
    except Exception as e:
        print(f"⚠️ Error fetching video for '{query}': {e}")

    return None  # Return None if no video found

# Function to make video vertical
def make_vertical(video_clip):
    target_size = (1080, 1920)  # Vertical 9:16 aspect ratio

    if video_clip.size[0] > video_clip.size[1]:
        # Crop sides if the video is horizontal
        new_width = int(video_clip.size[1] * 9 / 16)
        video_clip = video_clip.crop(x_center=video_clip.size[0] // 2, width=new_width)
    else:
        video_clip = video_clip.resize(height=1920)

    return video_clip.set_position("center").resize(target_size)

# Function to generate full vertical video
def create_full_video(storyboard):
    video_clips = []

    for index, scene in enumerate(storyboard):
        print(f"🎬 Processing: {scene['scene']}")

        # Generate TTS Audio
        audio_filename = f"audio/audio_{index}.mp3"
        try:
            tts = gTTS(scene["audio"], lang='en')
            tts.save(audio_filename)
            audio_clip = AudioFileClip(audio_filename)
            scene_duration = audio_clip.duration
        except Exception as e:
            print(f"⚠️ Error generating audio for scene {index}: {e}")
            continue  # Skip this scene if audio fails

        # Fetch relevant video from Pexels
        video_url = fetch_pexels_video(scene["visual"])
        if video_url:
            video_filename = f"videos/video_{index}.mp4"
            try:
                video_response = requests.get(video_url)
                with open(video_filename, "wb") as f:
                    f.write(video_response.content)

                video_clip = VideoFileClip(video_filename).subclip(0, min(scene_duration, 5))
                video_clip = make_vertical(video_clip)  # Ensure vertical orientation
            except Exception as e:
                print(f"⚠️ Error processing video for scene {index}: {e}")
                video_clip = None
        else:
            video_clip = None

        # If no video is found, use a text-based fallback clip
        if not video_clip:
            video_clip = TextClip(scene["visual"], fontsize=50, color='white', bg_color='black', size=(1080, 1920)).set_duration(scene_duration)

        # Set audio for the clip
        video_clip = video_clip.set_audio(audio_clip)

        # Append to list
        video_clips.append(video_clip)

        # Delay to avoid API throttling
        time.sleep(1)

    if not video_clips:
        print("❌ No valid scenes processed. Exiting.")
        return

    # Combine all video clips
    final_video = concatenate_videoclips(video_clips, method="compose")

    # Save final video
    output_filename = "final_vertical_story_video.mp4"
    final_video.write_videofile(output_filename, codec="libx264", audio_codec="aac")

    print(f"✅ Final vertical video saved as: {output_filename}")

# Main script
parsed_storyboard = parse_storyboard(storyboard_text)

if parsed_storyboard:
    create_full_video(parsed_storyboard)
else:
    print("❌ No valid storyboard data found.")


🎬 Processing: Scene 1
🎬 Processing: Scene 2
🎬 Processing: Scene 3
🎬 Processing: Scene 4
🎬 Processing: Scene 5
🎬 Processing: Scene 6
🎬 Processing: Scene 7
🎬 Processing: Scene 8
Moviepy - Building video final_vertical_story_video.mp4.
MoviePy - Writing audio in final_vertical_story_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_vertical_story_video.mp4



Moviepy - Done !
Moviepy - video ready final_vertical_story_video.mp4
✅ Final vertical video saved as: final_vertical_story_video.mp4
